# Desarrollo de la Interfaz Streamlit (ui.py)
Este notebook documenta el desarrollo y funcionamiento de la interfaz creada con Streamlit, que hemos utilizada para la visualización y análisis del dataset `AI4I 2020 Predictive Maintenance`.

La aplicación desarrollada tiene dos funciones principales:

1. **Exploración de datos**  
   - KPIs generales  
   - Distribuciones de fallos  
   - Visualizaciones interactivas  
   - Correlaciones y estadísticas  

2. **Predicciones**  
   - Ser capaz de utilizar en tiempo real las predicciones creadas mediante `BentoML` y desplegadas en nuestra interfaz de `Sreamlit`.
   - Realizar comparativas a tiempo real de el rendimiento de nuestros modelos.


### Estructura general de la interfaz

En esta sección se describe el proceso de carga de datos utilizado dentro de Streamlit y la estructura global del archivo `ui.py`.

Para asegurar un rendimiento óptimo en Streamlit se emplea `@st.cache_data`, que evita recargar los datos innecesariamente.

Tambien se definen las columnas mas relevantes:
- **numeric_cols**: Lista de columnas continuas que representan sensores de la máquina
- **fallos_cols**: Lista de columnas que representan distintos tipos de fallos

La aplicación está dividida en dos secciones mediante un menú lateral:
- **Exploración de datos**
- **Predicciones**

Cada sección muestra contenido dinámico según la selección del usuario.

Se presenta un título principal y una descripción detallada del dataset y las variables que contiene para dar contexto al usuario.  


In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import plotly.express as px
import numpy as np

# Configuración de la página
st.set_page_config(
    page_title="Mantenimiento Predictivo WAI4I 2020",
    layout="wide"
)

# Cargar datos
@st.cache_data
def load_data():
    df = pd.read_csv(r"C:\Users\Usuario\Documents\GitHub\ADDPLI---Proyecto-Final\data\ai4i2020.csv")  
    return df

df = load_data()

# Columnas continuas y de fallos
numeric_cols = ["Air temperature [K]", "Process temperature [K]", "Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]
fallos_cols = ["TWF","HDF","PWF","OSF","RNF"]

# Calcular conteo total de fallos
fallos_count = df[fallos_cols].sum()

# Sidebar para navegación
st.sidebar.title("Navegación")
opcion = st.sidebar.radio("Selecciona una sección:", ["Exploración de datos", "Predicciones"])

# Contenido principal
st.title("Análisis y Predicción de Fallos ")
st.markdown("""
Este panel utiliza el dataset **WAI4I 2020**, que contiene datos de sensores de máquinas industriales para analizar y predecir fallos.  
Incluye variables como temperatura del aire y del proceso, velocidad de rotación, torque y desgaste de la herramienta, así como registros de distintos tipos de fallos:  

- **TWF:** Desgaste de herramienta  
- **HDF:** Disipación de calor  
- **PWF:** Potencia fuera de rango  
- **OSF:** Sobreesfuerzo mecánico  
- **RNF:** Falla aleatoria  

Cada registro corresponde a una máquina en un momento determinado. Con esta información podemos explorar patrones, estudiar relaciones entre variables y desarrollar modelos de mantenimiento predictivo.
""")

2025-12-14 09:02:13.432 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:13.432 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-12-14 09:02:13.441 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-12-14 09:02:13.441 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:14.021 
  command:

    streamlit run c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-14 09:02:14.021 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:14.029 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored wh

DeltaGenerator()

## EXPLORACION DE DATOS

### Colores para visualizaciones

Se define una paleta de colores para cada tipo de fallo.  
Esto permite que los gráficos sean consistentes y fáciles de interpretar visualmente.

### KPIs Generales

Se calculan indicadores clave (KPIs) para resumir el estado general de los fallos:

- **Total de fallos:** suma de todos los registros de fallos en el dataset.
- **Tipo de fallo más frecuente:** el fallo que ocurre con mayor frecuencia.
- **Máquinas con al menos un fallo:** número de máquinas que presentan algún tipo de fallo.

In [ ]:
# Exploración de datos
if opcion == "Exploración de datos":
    
    fallos_colors = ["#4CAF50", "#FFC107", "#F44336", "#2196F3", "#9C27B0"]
    fallos_color_scale = alt.Scale(domain=fallos_cols, range=fallos_colors)

    # KPIs generales
    st.subheader("KPIs Generales")
    total_fallos = df[fallos_cols].sum().sum()
    tipo_mas_frecuente = df[fallos_cols].sum().idxmax()
    cantidad_mas_frecuente = df[fallos_cols].sum().max()
    num_maquinas_fallo = df[df[fallos_cols].sum(axis=1) > 0]["Product ID"].nunique()
    
    kpi1, kpi2, kpi3 = st.columns(3)
    kpi1.metric("Total de fallos", f"{total_fallos}")
    kpi2.metric("Tipo de fallo más frecuente", f"{tipo_mas_frecuente} ({cantidad_mas_frecuente})")
    kpi3.metric("Máquinas con al menos un fallo", f"{num_maquinas_fallo}")

    st.markdown("---")

2025-12-14 09:02:20.535 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:20.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:20.538 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:20.547 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:20.547 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:20.547 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:20.560 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:20.561 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

### Distribución de fallos y fallos por máquina

Dividimos la pantalla en **dos columnas** para mostrar diferentes gráficos al mismo tiempo:

- Columna 1: Pie chart de los tipos de fallos.  
    - Creamos un DataFrame con la cantidad y porcentaje de cada tipo de fallo.  
    - Generamos un gráfico que muestran el nombre y la cantidad de cada fallo.  
    - Se utiliza la paleta de colores definida anteriormente para mantener consistencia visual.

- Columna 2: Gráfico de barras apiladas mostrando fallos por tipo de máquina.
    - Se clasifica la primera letra del `Product ID` como `MachineType`.  
    - Tranformamos el DataFrame para que cada fila corresponda a un tipo de fallo por máquina.  
    - Ponemos un **selectbox** para filtrar por tipo de máquina y generamos un gráfico de barras apiladas mostrando la cantidad de cada tipo de fallo por máquina.  
    - Ademas, tooltips permiten ver los valores exactos al pasar el mouse sobre cada barra.

In [4]:
    # Crear columnas para gráficos lado a lado
    col1, col2 = st.columns(2)

    # Pie chart de fallos con porcentajes en col1
    with col1:
        st.subheader("Distribución de los tipos de fallos")
        fallos_df = pd.DataFrame({
            "Tipo de fallo": fallos_cols,
            "Cantidad": df[fallos_cols].sum().values
        })
        fallos_df["Porcentaje"] = (fallos_df["Cantidad"] / fallos_df["Cantidad"].sum() * 100).round(1)
        fallos_df["label"] = fallos_df["Tipo de fallo"] + " (" + fallos_df["Porcentaje"].astype(str) + "%)"

        fallos_colors = ["#4CAF50", "#FFC107", "#F44336", "#2196F3", "#9C27B0"]
        chart = alt.Chart(fallos_df).mark_arc(innerRadius=50).encode(
            theta=alt.Theta(field="Cantidad", type="quantitative"),
            color=alt.Color(field="Tipo de fallo", type="nominal", scale=alt.Scale(range=fallos_colors)),
            tooltip=["label:N", "Cantidad:Q"]
        )
        st.altair_chart(chart, use_container_width=True)
       

    # Gráfico interactivo: fallos vs variable seleccionada en col2
    # Gráfico interactivo mejorado: fallos vs variable seleccionada en col2
    with col2:
        st.subheader("Cantidad de fallos por tipo de máquina")
        df["MachineType"] = df["Product ID"].str[0]
        fail_long = (
            df.groupby(["MachineType"])[fallos_cols]
            .sum()
            .reset_index()
            .melt(id_vars="MachineType", var_name="Tipo de fallo", value_name="Cantidad")
        )

        # Filtro por máquina
        selected_machine = st.selectbox("Filtra por tipo de máquina:", ["Todas"] + sorted(df["MachineType"].unique()))
        if selected_machine != "Todas":
            fail_filtered = fail_long[fail_long["MachineType"] == selected_machine]
        else:
            fail_filtered = fail_long

        color_scale = alt.Scale(domain=fallos_cols, range=fallos_colors)
        stacked_bar = (
            alt.Chart(fail_filtered)
            .mark_bar()
            .encode(
                x=alt.X("Tipo de fallo:N", title="Tipo de fallo"),
                y=alt.Y("Cantidad:Q", title="Cantidad total"),
                color=alt.Color("Tipo de fallo:N", scale=color_scale),
                tooltip=["Tipo de fallo:N", "Cantidad:Q"]
            )
            .properties(width=500, height=350)
        )
        st.altair_chart(stacked_bar, use_container_width=True)

    st.markdown("---")

2025-12-14 09:02:25.487 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:25.487 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:25.487 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:25.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:25.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:25.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:02:25.568 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'` o

DeltaGenerator()

### Visualización adicional de fallos y matriz de co-ocurrencia

Dividimos la pantalla de nuevo en **dos columnas**:

- Columna 3: Gráfico de líneas (similar a boxplot) que relaciona una variable continua con la cantidad de fallos.
    - Permite analizar cómo se distribuyen los fallos según una variable continua seleccionada.  
    - Se divide la variable en **bins** para agrupar los datos y se calcula la suma de fallos en cada intervalo.  
    - Creamos un **gráfico de líneas con puntos**, donde cada punto representa la cantidad de fallos en un bin.  
    - `st.selectbox` Permite elegir la variable y el tipo de fallo, y `st.slider` ajusta la cantidad de bins.
- Columna 4: Matriz de co-ocurrencia de fallos para identificar relaciones entre distintos tipos de fallos.
    - Se calcula la cantidad de veces que **dos tipos de fallos ocurren juntos** en un mismo registro y se genera un heatmap interactivo donde el color indica la frecuencia de co-ocurrencia.  
    - Esto permite identificar patrones o relaciones entre distintos tipos de fallos.

In [6]:
# Aquí se pueden hacer más columnas
col3, col4 = st.columns(2)

    # Boxplots en col3
with col3:
    st.subheader("Cantidad de fallos según variable seleccionada")
    selected_var = st.selectbox("Selecciona la variable X:", numeric_cols)
    selected_fail = st.selectbox("Selecciona el tipo de fallo:", fallos_cols)
    num_bins = st.slider("Número de bins:", min_value=5, max_value=50, value=10)

    df_plot = df[[selected_var, selected_fail]].copy()
    df_plot["bin"] = pd.cut(df_plot[selected_var], bins=num_bins)
    fail_counts = df_plot.groupby("bin")[selected_fail].sum().reset_index()
    fail_counts["bin_str"] = fail_counts["bin"].astype(str)

        # Gráfico de líneas
    line_fail = alt.Chart(fail_counts).mark_line(point=True, color="salmon").encode(
        x=alt.X("bin_str:N", title=selected_var),
        y=alt.Y(f"{selected_fail}:Q", title=f"Cantidad de {selected_fail}"),
        tooltip=["bin_str", selected_fail]
    ).properties(width=500, height=300)

    st.altair_chart(line_fail)
        

    # Histogramas en col4
with col4:
    st.subheader("Matriz de co-ocurrencia de fallos")
    co_occur = df[fallos_cols].T.dot(df[fallos_cols])
    fig = px.imshow(co_occur, text_auto=True, color_continuous_scale='Blues', width=1000, height=600)
    fig.update_xaxes(side="top")  
    st.plotly_chart(fig, use_container_width=True)

st.markdown("---")

2025-12-14 09:07:07.239 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:07.242 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:07.242 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:07.247 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:07.247 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:07.251 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:07.253 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:07.255 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

### Correlaciones, distribuciones y otras estadísticas

1. Heatmap de correlaciones
    - Calculamos la **correlación** entre las variables continuas usando `df[numeric_cols].corr()` y generamos un mapa de calor donde cada celda indica el coeficiente de correlación entre dos variables.   

2. Histograma de variables continuas
    - Permite visualizar la **distribución de los valores** de cada una de las variable continuas.
    - El usuario eleje la variable a visualizar
    - Creamos un histograma y añadimos la estimación de densidad de probabilidad. 

3. Datos y estadísticas descriptivas
    - Permitimos al usuario decidir si quiere ver la tabla de datos o las estadísticas.
    - `df.head()` muestra los **primeros registros** del dataset.  
    - `df.describe()` muestra estadísticas como media, desviación estándar, valores mínimo y máximo, cuartiles, etc. 

In [9]:
with st.expander("Correlación de variables continuas"):
    st.write("Heatmap de correlaciones entre variables numéricas")
    corr = df[numeric_cols].corr()
    fig, ax = plt.subplots(figsize=(6,5))
    sns.heatmap(corr, annot=True, cmap="coolwarm", ax=ax)
    st.pyplot(fig)

with st.expander("Distribución de variables continuas"):
    st.write("Selecciona la variable para ver su histograma")
    selected_numeric = st.selectbox("Variable:", numeric_cols)
        
    fig, ax = plt.subplots(figsize=(5,4))  
    sns.histplot(df[selected_numeric], kde=True, bins=30, color="#4CAF50")
    ax.set_title(selected_numeric)
    st.pyplot(fig)
        
with st.expander("Datos y estadísticas"):
    if st.checkbox("Mostrar tabla de datos"):
        st.dataframe(df.head())
    if st.checkbox("Mostrar estadísticas descriptivas"):
        st.dataframe(df.describe())

2025-12-14 09:07:41.669 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:41.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:41.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:41.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:41.867 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:42.010 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:42.010 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-14 09:07:42.010 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

## Predicciones

La segunda sección principal de la aplicación se centra en las predicciones, cuyo objetivo es permitir al usuario interactuar en tiempo real con los modelos de Machine Learning desplegados mediante BentoML.

Esta pestaña conecta la interfaz Streamlit con una API REST local, donde se encuentran servidos los modelos previamente entrenados.

El flujo general de esta sección es el siguiente:

1. Configuración de la conexión con BentoMl

2. Selección del modelo de predicción

3. Ingreso manual de los parámetros de entrada

4. Envío de los datos a la API de BentoML

5. Recepción e interpretación de la predicción y visualización del resultado

6. Evaluación y comparacion de modelos

### Configuración de la conexión con BentoMl

In [ ]:
import requests

# URL base donde se ejecuta el servicio BentoML
BENTO_URL_BASE = "http://localhost:3000"

def call_bento_api_raw(endpoint_name: str, payload: dict) -> dict:
    """
    Realiza una petición POST al endpoint especificado de BentoML
    y devuelve la respuesta en formato JSON.
    """
    url = f"{BENTO_URL_BASE}/{endpoint_name}"
    try:
        response = requests.post(
            url,
            json=payload,
            timeout=10
        )
        response.raise_for_status()
        return response.json()
    except Exception as e:
        return {"error": str(e)}


## 

### Selección del modelo de predicción

Este bloque define los modelos de Machine Learning disponibles en la aplicación y los relaciona con sus respectivos endpoints en BentoML.
Mediante un selector interactivo, el usuario puede elegir el modelo a utilizar, y la aplicación determina automáticamente qué servicio de la API debe ser invocado.


In [ ]:
elif opcion == "Predicciones":
    st.header("Predicción de Fallos en Tiempo Real")
    st.markdown("Consulta la API de BentoML con nuevos parámetros. El servicio se ejecuta en http://localhost:3000.")
        # 1. Selector de Modelo y su endpoint 
    MODEL_ENDPOINTS = {
        "XGBoost (Algoritmo de Clasificación)": "predict_xgb", 
        "Regresión Logística (Algoritmo de Regresión)": "predict_logreg",
        "SVM (Algoritmo de Clasificación)": "predict_svm",
        "Random Forest (Algoritmo de Clasificación)": "predict_random_forest", 
        "HDBSCAN (Algoritmo de Agrupación)": "cluster_hdbscan",
    }
    selected_model_display = st.selectbox(
        "Modelo a utilizar:", 
        list(MODEL_ENDPOINTS.keys())
    )
    endpoint_to_call = MODEL_ENDPOINTS[selected_model_display]

    # Determinar cuántas columnas/features necesita el modelo
    if endpoint_to_call in ["predict_xgb", "predict_logreg", "predict_svm"]:
        required_cols = 12
    else:
        required_cols = 7



### Ingreso manual de los parámetros de entrada

En esta sección se construye un formulario interactivo que permite al usuario introducir manualmente los valores de las variables de entrada.
Se incluyen tanto variables continuas, que representan los sensores de la máquina, como variables categóricas codificadas mediante dummies.
El uso de un formulario garantiza que todos los parámetros se envíen de manera conjunta, evitando ejecuciones parciales o inconsistentes.



In [ ]:
    st.subheader(f"Ingreso de Parámetros ({required_cols} Features Requeridas)")

    with st.form("prediction_form"):
        col_t1, col_t2 = st.columns(2)

        # INPUTS Continuos
        with col_t1:
            st.markdown("*Variables Continuas:*")
            temp_aire = st.number_input("Air temperature [K]", value=299.1, step=0.1)
            temp_proceso = st.number_input("Process temperature [K]", value=309.2, step=0.1)
            velocidad = st.number_input("Rotational speed [rpm]", value=1530, step=1)
            torque = st.number_input("Torque [Nm]", value=40.1, step=0.1)
            desgaste = st.number_input("Tool wear [min]", value=100, step=1)
        
        # INPUTS Dummies
        with col_t2:
            st.markdown("*Tipo de Producto:*")
            machine_type = st.radio("Selecciona Tipo:", ('L', 'M', 'H'), index=0, key="machine_type_radio")
            
            type_L = 1 if machine_type == 'L' else 0
            type_M = 1 if machine_type == 'M' else 0

            # Inicialización de los fallos
            twf, hdf, pwf, osf, rnf = 0, 0, 0, 0, 0
            
            if required_cols == 12:
                st.markdown("*Fallos Históricos (5 Dummies):*")
                twf = st.checkbox("TWF (Tool Wear Failure)", value=False)
                hdf = st.checkbox("HDF (Heat Dissipation Failure)", value=False)
                pwf = st.checkbox("PWF (Power Failure)", value=False)
                osf = st.checkbox("OSF (Overstrain Failure)", value=False)
                rnf = st.checkbox("RNF (Random Failure)", value=False)
            else:
                st.info("El modelo de 7 Features solo utiliza las variables continuas y el tipo de máquina (L, M).")
        
        submitted = st.form_submit_button("Obtener Predicción")


### Envío de los datos a la API de BentoML

Una vez enviados los datos por el usuario, se construye el vector de características respetando el mismo orden utilizado durante el entrenamiento de los modelos.
Posteriormente, los datos se encapsulan en un objeto JSON y se envían mediante una petición HTTP POST a la API de BentoML.
Este paso conecta la interfaz Streamlit con los modelos desplegados en producción.

In [ ]:
        if submitted:
            st.warning("Verificando el orden de las features...")

            if required_cols == 12:
                input_features = [
                    temp_aire,
                    temp_proceso,
                    velocidad,
                    torque,
                    desgaste,
                    int(twf),
                    int(hdf),
                    int(pwf),
                    int(osf),
                    int(rnf),
                    type_L,
                    type_M,
                ]
            else:
                input_features = [
                    temp_aire, temp_proceso, velocidad, torque, desgaste,
                    type_L, type_M
                ]
            
            # Función que llama a la API con payload completo (input_obj)
            def call_bento_api_raw(endpoint_name: str, payload: dict) -> dict:
                url = f"{BENTO_URL_BASE}/{endpoint_name}"
                try:
                    response = requests.post(
                        url,
                        json=payload,
                        timeout=10
                    )
                    response.raise_for_status()
                    return response.json()
                except Exception as e:
                    return {"error": str(e)}

            # Preparación del  payload en el formato que espera BentoML
            payload = {
                "input_obj": {
                    "input_data": [input_features]
                }
            }

            st.info(f"Conectando con la API y usando el modelo: *{selected_model_display}*...")
            result = call_bento_api_raw(endpoint_to_call, payload)




### Recepción e interpretación de la predicción y visualización del resultado 

En este bloque se procesa la respuesta devuelta por la API.
Dependiendo del tipo de modelo seleccionado (clasificación binaria, multi-etiqueta o clustering), la predicción se interpreta de forma distinta:
como una clase, una probabilidad, un conjunto de fallos o un identificador de cluster.
Este tratamiento diferencial permite manejar distintos enfoques de aprendizaje dentro de una misma interfaz.

Los resultados se adaptan al tipo de modelo utilizado, mostrando advertencias en caso de fallo previsto o confirmando una operación normal.


In [ ]:
#Mostrar resultados            
            if "error" in result:
                st.error(f"Fallo en la conexión o la API: {result['error']}")
                st.code(f"Error de la API: {result['error']}")
            else:
                pred_array = np.array(result)
                
                st.subheader("Resultado de la Predicción:")

                if "cluster" in endpoint_to_call or pred_array.shape[-1] == 1:
                    prediction = int(pred_array.flatten()[0])
                    label = "Cluster Asignado" if "cluster" in endpoint_to_call else "Clase Predicha (0=Normal, 1=Fallo)"
                    st.metric(label=label, value=prediction)
                    
                    if prediction == 1:
                        st.error("*ALERTA: FALLO PREVISTO* (Clase 1)")
                    elif prediction == 0:
                        st.success("*Operación Normal* (Clase 0)")
                    elif prediction == -1 and "cluster" in endpoint_to_call:
                        st.warning("Patrón Atípico (Ruido -1).")

                elif pred_array.shape[-1] == 2:
                    prob_fail = float(pred_array.flatten()[1])
                    st.metric("Probabilidad de Fallo (Clase 1)", f"{prob_fail:.2%}")

                    if prob_fail > 0.5:
                        st.error("*ALERTA: FALLO PREVISTO* (Probabilidad > 50%)")
                    else:
                        st.success("Operación Normal (Probabilidad <= 50%)")

                elif "random_forest" in endpoint_to_call:
                    st.markdown("##### Resultados Multi-Etiqueta (Random Forest)")
                    
                    fallo_predicho = pred_array[0]
                    fallos_cols_rf = ["TWF","HDF","PWF","OSF","RNF"] 
                    
                    fallos_df_pred = pd.DataFrame([fallo_predicho], columns=fallos_cols_rf)

                    if fallos_df_pred.sum(axis=1).iloc[0] == 0:
                        st.success("*Predicción: Ningún fallo específico*")
                    else:
                        st.error("*Fallo(s) detectado(s)*")
                        fallos_activos = fallos_df_pred.columns[fallos_df_pred.iloc[0] == 1].tolist()
                        st.code(f"Tipos de fallo predichos: {', '.join(fallos_activos)}")

                else:
                    st.json(result)


### Evaluación y comparación de modelos

Después de realizar predicciones en tiempo real con los diferentes modelos, es fundamental evaluar su desempeño global para comprender cuál es más adecuado para predecir fallos en la máquina. La evaluación se realiza en dos niveles:

1. Clasificación Binaria: Predicción de Fallo General

En esta sección se comparan los modelos que predicen si un proceso de manufactura resultará en un fallo o no (Machine Failure, 0 o 1). Los modelos evaluados incluyen XGBoost, Regresión Logística y SVM.

Se presentan métricas clave:

- Accuracy: proporción de predicciones correctas sobre el total de registros.

- Precision: porcentaje de predicciones de fallo correctas sobre todas las predicciones de fallo.

- Recall: capacidad del modelo para identificar correctamente los fallos reales.

- F1 Score: medida que combina precisión y recall.

- AUC (ROC): área bajo la curva ROC, indica la capacidad discriminatoria del modelo.

Estas métricas permiten identificar no solo qué modelo es más preciso, sino también cuál minimiza los falsos negativos, algo crítico en mantenimiento predictivo. En nuestro caso, XGBoost destaca por su robustez y alto recall, asegurando que la mayoría de los fallos reales sean detectados.

2. Análisis del Mejor Modelo

Se profundiza en el mejor modelo seleccionado, XGBoost, mostrando:

- Matriz de Confusión: distribución de verdaderos y falsos positivos/negativos, que ayuda a visualizar dónde ocurren errores de clasificación.

- Curva ROC y AUC: confirma su alta capacidad para distinguir entre operaciones normales y fallos.

- Estas visualizaciones permiten interpretar de manera clara el desempeño del modelo y justificar su uso en producción.

3. Evaluación de Random Forest para Clasificación Multi-Etiqueta

Mientras que XGBoost y otros modelos se centran en predecir si habrá un fallo, Random Forest se utiliza para identificar el tipo específico de fallo (TWF, HDF, PWF, OSF, RNF).

Se presentan métricas ponderadas:

- Accuracy Total: porcentaje general de predicciones correctas.

- Precision Ponderada: precisión considerando la distribución de cada tipo de fallo.

- Recall Ponderado: capacidad de identificar correctamente cada tipo de fallo.




In [ ]:
# Evaluación y comparación entre nuestros modelos
    st.header("Evaluación y Comparativa de Modelos")
    st.markdown("Análisis de las métricas clave y la justificación del mejor modelo para la predicción de fallos.")

    # 1. Tabla de Métricas de Clasificación Binaria (Fallo General)
    st.markdown("### 1. Métricas de Modelos de Clasificación Binaria")
    st.write("Métricas de los modelos que predicen 'Machine Failure' (Clase 0 o 1).")

    # Datos REALES (Extraídos del notebook)
    data_clasificacion = {
        'Modelo': ["XGBoost", "Regresión Logística", "SVM"],
        'Accuracy': [0.9990, 0.9990, 0.9990],
        'Precision': [1.0000, 1.0000, 1.0000], 
        'Recall': [0.9672, 0.9672, 0.9672],
        'F1 Score': [0.9833, 0.9833, 0.9833],
        'AUC (ROC)': [0.9990, 0.9990, 0.9990]
    }

    df_metricas = pd.DataFrame(data_clasificacion)

    # Resaltar la mejor métrica en cada columna
    st.dataframe(
        df_metricas.style.highlight_max(
            subset=['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC (ROC)'], 
            axis=0, 
            props='font-weight: bold; background-color: #d8f5d8; color: #000000;'
        ).format(precision=4), 
        use_container_width=True
    )

    st.info("""
    *Conclusión sobre la Predicción Binaria:*
    Todos los modelos son excepcionalmente buenos, indicando que las features preprocesadas son muy predictivas. 
    Se elige *XGBoost* por su reconocida robustez en producción. El *Recall (0.9672)* es vital ya que minimiza los Falsos Negativos (fallos reales no detectados).
    """)

    # 2. Justificación y Visualizaciones del Mejor Modelo
    mejor_modelo_nombre = "XGBoost" 
    st.markdown(f"### 2. Análisis del Mejor Modelo: *{mejor_modelo_nombre}*")

    col_conf, col_roc = st.columns(2)

    with col_conf:
        st.markdown("#### Matriz de Confusión")
        st.write(f"Distribución de True/False Positives/Negatives para {mejor_modelo_nombre}.")
        

        try:
            st.image("img/xgb_confusion.png", caption=f"Matriz de Confusión de {mejor_modelo_nombre}") 
        except Exception:
            st.warning("No se encontró la imagen 'img/xgb_confusion.png'. Asegúrate de que está en la carpeta 'img'.")
            
    with col_roc:
        st.markdown("#### Curva ROC y Área bajo la Curva (AUC)")
        st.write(f"El valor de AUC de {data_clasificacion['AUC (ROC)'][0]:.4f} confirma su alta capacidad discriminatoria.")
        
    
        try:
            st.image("img/xgb_rocauc.png", caption="Curva ROC de XGBoost") 
        except Exception:
            st.warning("No se encontró la imagen 'img/xgb_rocauc.png'. Asegúrate de que está en la carpeta 'img'.")

    # 3. Evaluación de Random Forest (Clasificación de Fallos Específicos) 
    st.markdown("### 3. Evaluación de Random Forest (Clasificación Multi-Etiqueta)")
    st.write("""
    El modelo Random Forest atiende a la pregunta *'Si hay un fallo, ¿cuál de los 5 tipos es?'*. Se evalúa con métricas ponderadas.
    """)

    # Datos REALES 
    rf_accuracy = 0.9800
    rf_precision = 0.6979
    rf_recall = 0.4722
    rf_f1 = 0.5523

    col_rf1, col_rf2, col_rf3 = st.columns(3)
    col_rf1.metric(label="Accuracy Total (RF)", value=f"{rf_accuracy:.2%}")
    col_rf2.metric(label="Precision Ponderada (RF)", value=f"{rf_precision:.2%}")
    col_rf3.metric(label="Recall Ponderado (RF)", value=f"{rf_recall:.2%}")

    st.warning(f"""
    *Análisis del Random Forest:*
    El *Recall Ponderado ({rf_recall:.2%})* es bajo, lo que indica que el modelo tiene dificultades para identificar correctamente los tipos de fallos específicos. Este modelo debe usarse solo para clasificar el tipo después de que un modelo binario (XGBoost) haya predicho que ocurrirá una falla.
    """)